# OCO-2 XCO2 Data Analysis

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import datetime
import matplotlib.pyplot as plt
# https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html
plt.style.use('seaborn-v0_8')

import numpy as np
import os
import os.path
import pandas as pd

from IPython.core.display import HTML
from IPython.display import clear_output
from IPython.core.pylabtools import figsize

In [3]:
from oco_tools import *

In [4]:
file_dict = dict_oco2_xco2
file_dict

{'lat': 'latitude',
 'lon': 'longitude',
 'sza': 'solar_zenith_angle',
 'vza': 'sensor_zenith_angle',
 'saa': 'Sounding/solar_azimuth_angle',
 'vaa': 'Sounding/sensor_azimuth_angle',
 'xco2': 'xco2',
 'xco2_sigma': 'xco2_uncertainty',
 'xco2_qual': 'xco2_quality_flag',
 'time': 'time'}

In [5]:
# which dataset
path_files = 'C:\\Users\\ASUS\\Downloads\\oco2\\oco-xco2\\data\\DBSH'


## Select region

In [6]:
# NZ bounding box
latMin = 8
latMax = 23
lonMin = 102
lonMax = 109

In [7]:
oco2_aoi = OCO2(path_files+'\\*.nc4', file_dict, latMin, latMax, lonMin, lonMax)

found 94 files
total num soundings in aoi  2477  total 15063540  % match 0.01644367791369094


In [8]:
t_unit_oco2    = "seconds since 1970-1-1 0:0:0"
t_cal = u"gregorian"

# Convert oco2 time units to a python time
oco2_aoi.time_python  = convert_time(oco2_aoi.time, t_unit_oco2, t_cal)
oco2_aoi.time_str = [datetime.datetime.fromisoformat(str(date)) for date in oco2_aoi.time_python]

In [9]:
# oco2_aoi.time[0] # seconds since 1970
# oco2_aoi.time_python[0].strftime("%d/%m/%Y, %H:%M:%S")
# oco2_aoi.time_str[0]

## Timeseries

In [10]:
# plot a timeseries
plt.figure(figsize=(20,6))
plt.subplot(1,2,1)
plt.plot(oco2_aoi.time_str, oco2_aoi.xco2,'.')
plt.ylabel('XCO2 (ppm)')
plt.title('VN 2024 CO2 Levels (OCO-2 XCO2)')
plt.savefig('images/oco2-xco2-vn-2024.png')

## Histogram

In [12]:
# let us look at a histogram
plt.figure(figsize=(10,6))
plt.hist(oco2_aoi.xco2, 100,range=(400,415), density=True, alpha=0.5, label='XCO2 (ppm)')
plt.title('VN 2024 CO2 Levels (OCO-2 XCO2)')
plt.legend(loc=0)
plt.savefig('images/oco2-xco2-vn-2024-hist.png')

In [13]:
print('Standard deviation of measurement ' + str(np.std(oco2_aoi.xco2)) )
print('Expected standard deviation based on posteriori error ' + str(np.mean(oco2_aoi.xco2_sigma)) )

Standard deviation of measurement 1.3290755
Expected standard deviation based on posteriori error 0.4757366


##  Running average timeseries

In [21]:
# Get data every 3 days and create a running mean of +/- 3 days:
# Define data range to smooth on (every 3 days)
from datetime import datetime
dates = np.arange('2020-01', '2020-06', dtype='datetime64[3D]').astype(datetime)

In [22]:
len(dates)

51

In [23]:
# Use dates with a 3 day time-step and now use a +/- 3 day filter for the running mean:
timeseries_oco2 = running_mean(oco2_aoi.time_str, oco2_aoi.xco2, dates, 3)

In [24]:
plt.figure(figsize=(15,6))
plt.errorbar(dates, timeseries_oco2.mean,yerr=timeseries_oco2.standard_error, label='OCO-2 XCO2 Mean')
plt.ylabel('XCO2 (ppm)')
plt.legend(loc=0)
plt.title('VN 2020 CO2 Levels (OCO-2 XCO2) +/-3 day running mean')
plt.savefig('images/oco2-xco2-vn-2020-ravg.png')

# fin